In [1]:
%load_ext autoreload
%autoreload 2
from readers.base import NeuralynxReader
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, decimate
import numpy as np
from tensorflow.keras.applications import EfficientNetB2
import pandas as pd
import stumpy
import cv2
from numba import cuda
import matplotlib.colors as mcolors
from detectors.motives import MotifFinder
%matplotlib tk

2022-04-30 15:23:15.952602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-30 15:23:15.952628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
rootdir, channel = '/media/sil2/Data/Lizard/Pogona Data/Lizard15/12.14.2015/18-26-12_cheetah', 32  # mark
video_file = f'{Path(rootdir).parent}/18-26-12_videos/2015-12-14_18-26-12-sub-10-fullFile--614316-Converted--1-614316.avi'

In [3]:
rd = NeuralynxReader(rootdir, channel=channel)
sc = rd.load_slow_cycles()
print(f'Reader initialized. fs={rd.fs:.0f}Hz, duration={rd.time_vector[-1]/3600:.1f} hours')

Reader initialized. fs=32000Hz, duration=17.1 hours


In [11]:
vid = cv2.VideoCapture(video_file)
pics = []
for _ in range(3):
  ret, frame = vid.read()
  gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (0, 0), fx=0.3, fy=0.3)
  pics.append(gray)

In [ ]:
descriptors = np.array([])
for pic in pics:
    kp, des = cv2.SIFT().detectAndCompute(pic, None)
    descriptors = np.append(descriptors, des)

desc = np.reshape(descriptors, (len(descriptors)/128, 128))
desc = np.float32(desc)
desc.shape

In [ ]:
del descriptors, des, kp, pic

In [20]:
sift = cv2.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)
des

array([[ 0.,  0.,  0., ...,  0.,  0., 30.],
       [ 0., 23., 79., ..., 28., 11.,  0.],
       [ 8., 52., 79., ..., 10.,  9.,  1.],
       ...,
       [ 3.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1., 15., 11., ...,  0.,  0.,  1.],
       [ 3.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)